# Robust Decision Making

To solve decision problems using bounded probability, we first need some code to calculate lower and upper expectations themselves.

## Credal Sets

Here, for simplicity, we will always specify a lower expectation by means of a finite set of probability mass functions.
The lower expectation is then the minimal expectation with respect to this set.

Let's move to lower expectations. A credal set will then be a set of probability mass functions. For simplicity, we will also use a sequence to represent these in the code. For example, when coding, we will represent the credal set $\{(0.2, 0.2, 0.6), (0.4, 0.1, 0.5)\}$ as follows:

In [1]:
# a credal set containing two probability mass functions
[[0.2, 0.2, 0.6], [0.4, 0.1, 0.5]]

[[0.2, 0.2, 0.6], [0.4, 0.1, 0.5]]

## Lower and Upper Expectations

The lower and upper expectation are simply the minimum and maximum expectation over the credal set $\mathcal{M}$:
$$\underline{E}(X)=\min_{p\in\mathcal{M}}E_p(X)\qquad\overline{E}(X)=\max_{p\in\mathcal{M}}E_p(X)$$
The next functions implement this.
We'll make use of the ``expectation`` function introduced previously.
In the code, we abstract the minimum and maximum transformations,
since we will consider additional transformation later.

In [2]:
from collections.abc import Callable, Sequence

PMF = Sequence[float]
Gamble = Sequence[float]

def expectation(pmf: PMF, gamble: Gamble) -> float:
    return sum(p * g for p, g in zip(pmf, gamble))

def transform_expectations(
    transform: Callable[[Sequence[float]], float],  # sequence of expectations -> float
    credal_set: Sequence[PMF],
    gamble: Sequence[Gamble],
) -> float:
    return transform([expectation(pmf, gamble) for pmf in credal_set])

def lower_expectation(credal_set: Sequence[Sequence[float]], gamble: Sequence[float]) -> float:
    return transform_expectations(min, credal_set, gamble)

def upper_expectation(credal_set: Sequence[Sequence[float]], gamble: Sequence[float]) -> float:
    return transform_expectations(max, credal_set, gamble)

Let's test this by finding the lower expectation of the gamble ``[5, 3, 1]`` with respect to the credal set ``[[0.2, 0.2, 0.6], [0.1, 0.1, 0.8]]``:

In [3]:
lower_expectation(
    credal_set=[[0.2, 0.2, 0.6], [0.1, 0.1, 0.8]],
    gamble=[5, 3, 1],
)

1.6

Let's check this value by evaluating the expectation of the gamble with respect to each member of the credal set:

In [4]:
[0.2 * 5 + 0.2 * 3 + 0.6 * 1, 0.1 * 5 + 0.1 * 3 + 0.8 * 1]

[2.2, 1.6]

The lower expectation is the lowest of these numbers, and indeed we can see that the correct value has been found.

**Exercise** Can you calculate the lower expectation of the gamble ``[1, 4, 2]`` with respect to the same credal set as the one used in the example?

In [5]:
# write your solution here

**Exercise** Can you also calculate the same gamble's upper expectation (again with the same credal set)?

In [6]:
# write your solution here

**Exercise** Verify that $\overline{E}(X)=-\underline{E}(-X)$, for the gamble ``[1, 4, 2]``.

In [7]:
# write your solution here

## Gamma-maximin

In $\Gamma$-maximin, we choose the gamble with the highest lower expectation:
$$\arg\max_{d\in D}\underline{E}(X_d)$$

In [8]:
def is_gamma_maxi_something(
    tol: float,  # absolute tolerance
    something: Callable[[Gamble], float],  # gamble -> float (e.g. lower prevision, upper prevision, ...)
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    values = list(map(something, gambles))
    max_value = max(values)
    return [value >= max_value - tol for value in values]

def is_gamma_maximin(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return lower_expectation(credal_set, gamble)
    return is_gamma_maxi_something(tol, something, gambles)

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [9]:
is_gamma_maximin(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[False, False, True]

This tells us that the third gamble (and only the third one) is optimal according to $\Gamma$-maximin.

## Gamma-maximax

The situation for $\Gamma$-maximax is very similar.
We choose the gamble with the highest upper expectation:
$$\arg\max_{d\in D}\overline{E}(X_d)$$
A small change in the above ``is_gamma_maximin`` function gives us ``is_gamma_maximax``:

In [10]:
def is_gamma_maximax(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return upper_expectation(credal_set, gamble)  # we changed just this line
    return is_gamma_maxi_something(tol, something, gambles)

Let us again test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [11]:
is_gamma_maximax(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, False, False]

This tells us that the first gamble (and only the first one) is optimal according to $\Gamma$-maximax.